## AWS Pipeline
This notebook reads in all the files in the Safegraph S3 bucket into the local raw data folder.

In [33]:
import os, sys
from dotenv import load_dotenv, find_dotenv
import boto3
from pathlib import Path
from loguru import logger
import datetime
import tempfile

from src import DATA_DIR

In [5]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

aws_access_key = os.environ.get("AWS_ACCESS_KEY")
aws_secret_access_key = os.environ.get("AWS_SECRET_ACCESS_KEY")
aws_bucket = os.environ.get("AWS_BUCKET")

In [24]:
# Initialize the session with authentication
session = boto3.Session(
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_access_key,
)

s3_client = session.client("s3", endpoint_url="https://s3.wasabisys.com")

In [35]:
AWS_FOLDERS = (
    "core/",
    "geo-supplement/",
    "monthly-patterns/normalization_stats",
    "monthly-patterns/patterns/",
    "monthly-patterns/patterns_backfill/2020/05/07/12/2019/",
    "monthly-patterns/patterns_backfill/2020/05/07/12/2020/",
    "open-census-data/",
)

In [36]:
def get_aws_files(folder):
    """Return the files in the s3 buckets as an iterator."""

    objects = s3_client.list_objects_v2(Bucket=aws_bucket, Prefix=folder)
    for response in objects["Contents"]:
        yield response

In [37]:
def download_with_progress_bar(local_path, remote_path):
    """Download a file from AWS and display a simple progress bar"""

    meta_data = s3_client.head_object(Bucket=aws_bucket, Key=remote_path)
    total_length = int(meta_data.get("ContentLength", 0))
    downloaded = 0

    def progress(chunk):
        nonlocal downloaded
        downloaded += chunk
        done = int(50 * downloaded / total_length)
        sys.stdout.write("\r[%s%s]" % ("=" * done, " " * (50 - done)))
        sys.stdout.flush()

    with open(local_path, "wb") as f:
        s3_client.download_fileobj(aws_bucket, remote_path, f, Callback=progress)

In [39]:
# Loop over each AWS folder we want to download
for aws_folder in AWS_FOLDERS:

    # Walk the AWS folder structure
    for response in get_aws_files(aws_folder):

        # Setup paths
        aws_path = Path(response["Key"])
        local_path = DATA_DIR / "raw" / aws_path

        # Download the new file if it doesn't exist or is out of date
        if (
            not local_path.exists()
            or datetime.utcfromtimestamp(local_path.stat().st_mtime).replace(
                tzinfo=timezone.utc
            )
            < response["LastModified"]
        ):

            # Log it
            logger.info(f"Processing {aws_path.name} from AWS...")

            # Setup local path
            if not local_path.parent.exists():
                local_path.parent.mkdir(parents=True)

            # Download
            download_with_progress_bar(str(local_path), str(aws_path))

2021-01-03 18:27:34.918 | INFO     | __main__:<module>:24 - Processing CoreRecords-CORE_POI-2019_03-2020-03-25.zip from AWS...


[==================================================]

2021-01-03 18:29:33.217 | INFO     | __main__:<module>:24 - Processing CoreApr2020Release-CORE_POI-2020_03-2020-04-07.zip from AWS...


[==================================================]

2021-01-03 18:32:10.929 | INFO     | __main__:<module>:24 - Processing CorePlacesMay2020Release-CORE_POI-2020_04-2020-05-06.zip from AWS...


...

[==================================================]

2021-01-03 20:55:08.498 | INFO     | __main__:<module>:24 - Processing patterns-part4.csv.gz from AWS...


[==================================================]

2021-01-03 20:55:35.717 | INFO     | __main__:<module>:24 - Processing safegraph_open_census_data.tar.gz from AWS...


[==================================================]

In [ ]:
aw